In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

/Users/apple/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
predictors = pd.read_excel('PredictorData2017.xlsx',sheet_name='Monthly',index_col=[0])

In [3]:
predictors.head()

,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,Rfree,infl,ltr,corpr,svar,csp,CRSP_SPvw,CRSP_SPvwx
yyyymm,,,,,,,,,,,,,,,,,
187101,4.44,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004955,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187102,4.50,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004514,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187103,4.61,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004243,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187104,4.74,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004632,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187105,4.86,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.003691,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
predictors = predictors.loc[195001:]

In [5]:
predictors.head()

,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,Rfree,infl,ltr,corpr,svar,csp,CRSP_SPvw,CRSP_SPvwx
yyyymm,,,,,,,,,,,,,,,,,
195001,17.05,1.15,2.33667,0.791417,0.0107,0.0257,0.0324,0.0215,0.027102,0.000892,-0.004237,-0.0061,0.0037,0.000989,0.004458,0.019703,0.018245
195002,17.22,1.16,2.35333,0.784998,0.0112,0.0258,0.0324,0.0214,0.025491,0.000933,0.000000,0.0021,0.0007,0.000426,0.004246,0.019603,0.009975
195003,17.29,1.17,2.37000,0.825528,0.0112,0.0258,0.0324,0.0215,0.029291,0.000933,0.004255,0.0008,0.0022,0.000624,0.004553,0.008185,0.003542
195004,18.07,1.18,2.42667,0.796497,0.0115,0.0260,0.0323,0.0214,0.026396,0.000958,0.000000,0.0030,-0.0008,0.000714,0.004022,0.045887,0.044493
195005,18.78,1.19,2.48333,0.761346,0.0116,0.0261,0.0325,0.0213,0.028571,0.000967,0.004237,0.0033,-0.0008,0.000641,0.003803,0.046902,0.037590


In [6]:
regdata=pd.DataFrame(index=predictors.index)

In [7]:
regdata['ret1m']=predictors['CRSP_SPvw'].shift(-1)-predictors['Rfree'].shift(-1)

In [8]:
regdata['ep'] = predictors['E12']/predictors['Index']

In [9]:
results = smf.ols('ret1m ~ ep', data=regdata).fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                  ret1m   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     3.028
Date:                Sun, 16 Sep 2018   Prob (F-statistic):             0.0822
Time:                        23:04:08   Log-Likelihood:                 1441.6
No. Observations:                 815   AIC:                            -2879.
Df Residuals:                     813   BIC:                            -2870.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0003      0.004      0.081      0.9

In [10]:
regdata['def']  = predictors['BAA'] - predictors['AAA']
regdata['term'] = predictors['lty'] - predictors['tbl']
regdata['ntis'] = predictors['ntis']
regdata.dropna(inplace=True)

In [11]:
regdata.head()

,ret1m,ep,def,term,ntis
yyyymm,,,,,
195001,0.018670,0.137048,0.0067,0.0108,0.027102
195002,0.007252,0.136663,0.0066,0.0102,0.025491
195003,0.044929,0.137073,0.0066,0.0103,0.029291
195004,0.045935,0.134293,0.0063,0.0099,0.026396
195005,-0.055182,0.132233,0.0064,0.0097,0.028571


In [12]:
X = regdata[['ep','def','term','ntis']]

In [13]:
Y = regdata['ret1m']

In [14]:
rho=X.corrwith(Y)

In [15]:
rho

ep      0.060918
def     0.017199
term    0.068031
ntis   -0.018166
dtype: float64

In [16]:
type(rho)

pandas.core.series.Series

In [17]:
rhokeep = rho.loc[np.abs(rho)>.1]

In [18]:
rhokeep

Series([], dtype: float64)

In [19]:
rhokeep.index

Index([], dtype='object')

In [20]:
Xsub=X[rhokeep.index]

In [21]:
Xsub.head()

""
yyyymm
195001
195002
195003
195004
195005


In [22]:
len(regdata)

815

In [23]:
window=120
regdata['KS_result']=0
regdata['CS_result']=0
X_=X.assign(constant=1).values
Y_=Y.values




In [24]:

for n in range (window, len(regdata)):
    
    x=X_[(n-window):n]
    y=Y_[(n-window):n]
    betas = np.linalg.inv(x.T.dot(x)).dot(x.T).dot(y)
    y_hat = betas.dot(X_[n, :])
    regdata['KS_result'].iloc[n] = y_hat
    
    

/Users/apple/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [25]:

for n in range (window, len(regdata)):
    x2=X[(n-window):n]
    y2=Y[(n-window):n]
    rho=x2.corrwith(y2)
    rhokeep=rho.loc[np.abs(rho)>0.1]
    xsub=x2[rhokeep.index]
    xsub_=xsub.assign(constant=1).values
    y2_=y2.values
    c_betas = np.linalg.inv(xsub_.T.dot(xsub_)).dot(xsub_.T).dot(y2_)
    #c_betas=c_betas[rhokeep.index]
    X2_=X[rhokeep.index].assign(constant=0).values
    c_y_hat = c_betas.dot(X2_[n, :])
    regdata['CS_result'].iloc[n] = c_y_hat 

/Users/apple/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [26]:
regdata.tail()


,ret1m,ep,def,term,ntis,KS_result,CS_result
yyyymm,,,,,,,
201707,0.001396,0.042521,0.0069,0.0155,-0.013104,0.013875,-0.004978
201708,0.019192,0.042911,0.0068,0.0141,-0.012138,0.015528,-0.004625
201709,0.023266,0.042503,0.0067,0.0156,-0.011027,0.014556,-0.004211
201710,0.030363,0.041943,0.0072,0.0154,-0.012358,0.014715,-0.004720
201711,0.010048,0.042148,0.0070,0.0137,-0.012243,0.020105,-0.004556


In [27]:
def weights(result):
    weight = np.minimum(1.5, np.maximum(-0.5, 100 * result))
    return weight

In [28]:
regdata['KS_weight'] = weights(regdata['KS_result'])
regdata['CS_weight'] = weights(regdata['CS_result'])
regdata['KS_ret'] = regdata['KS_weight'] * regdata['ret1m']
regdata['CS_ret'] = regdata['CS_weight'] * regdata['ret1m']

In [39]:
regdata

,ret1m,ep,def,term,ntis,KS_result,CS_result,KS_weight,CS_weight,KS_ret,CS_ret
yyyymm,,,,,,,,,,,
195001,0.018670,0.137048,0.0067,0.0108,0.027102,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
195002,0.007252,0.136663,0.0066,0.0102,0.025491,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
195003,0.044929,0.137073,0.0066,0.0103,0.029291,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
195004,0.045935,0.134293,0.0063,0.0099,0.026396,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
195005,-0.055182,0.132233,0.0064,0.0097,0.028571,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000
195006,0.016014,0.143584,0.0066,0.0101,0.030395,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
195007,0.047063,0.145740,0.0067,0.0098,0.030116,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
195008,0.058383,0.144408,0.0062,0.0094,0.029550,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
195009,0.003992,0.139846,0.0057,0.0090,0.025914,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [42]:
regdata=regdata[120:]

In [41]:
regdata.head()

,ret1m,ep,def,term,ntis,KS_result,CS_result,KS_weight,CS_weight,KS_ret,CS_ret
yyyymm,,,,,,,,,,,
196001,0.010622,0.060960,0.0073,0.0006,0.022116,-0.003744,-0.001745,-0.374428,-0.174526,-0.003977,-0.001854
196002,-0.015324,0.060406,0.0078,0.0033,0.024038,-0.000083,-0.001001,-0.008257,-0.100057,0.000127,0.001533
196003,-0.018805,0.061258,0.0076,0.0080,0.025593,0.004234,0.005727,0.423351,0.572652,-0.007961,-0.010769
196004,0.030771,0.061554,0.0075,0.0103,0.025576,0.007139,0.009529,0.713907,0.952918,0.021968,0.029323
196005,0.019752,0.059168,0.0082,0.0088,0.024413,0.007222,0.007572,0.722157,0.757221,0.014264,0.014957


In [43]:
KS_Mean = regdata['KS_ret'].mean() * 12
KS_Std = regdata['KS_ret'].std() * np.sqrt(12)
KS_Sharpe = regdata['KS_ret'].mean()/regdata['KS_ret'].std() * np.sqrt(12)
CS_Mean = regdata['CS_ret'].mean() * 12
CS_Std = regdata['CS_ret'].std() * np.sqrt(12)
CS_Sharpe = regdata['CS_ret'].mean()/regdata['KS_ret'].std() * np.sqrt(12)

In [44]:
compar={'KS_Mean':KS_Mean,
'KS_Std':KS_Std,
'KS_Sharpe':KS_Sharpe,
'CS_Mean': CS_Mean,
'CS_Std':CS_Std,
'CS_Sharpe':CS_Sharpe}

In [45]:
compar

{'CS_Mean': 0.04241971975016823,
 'CS_Sharpe': 0.37075117166601984,
 'CS_Std': 0.16863484549353899,
 'KS_Mean': 0.04853729158619175,
 'KS_Sharpe': 0.42421915635132101,
 'KS_Std': 0.11441560537637564}